In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-3.5-turbo", temperature=0)
response = llm.invoke("Hola, ¿cómo estás?")
response.text

'¡Hola! Estoy bien, gracias. ¿Y tú? ¿Cómo estás?'

In [3]:
response = llm.invoke("¿como esta el clima en Morelia Michoacan?")
response.text

'Actualmente en Morelia, Michoacán, el clima es soleado con una temperatura de alrededor de 25 grados Celsius. Se espera que el día continúe soleado y caluroso, con una ligera brisa en la tarde. No se esperan lluvias para el día de hoy.'

In [4]:
response = llm.invoke("Dime los productos que tienes en la tienda")
response.text

'Lo siento, no tengo una lista específica de productos en la tienda ya que soy un asistente virtual y no tengo acceso a esa información. ¿Hay algo más en lo que pueda ayudarte?'

In [ ]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Get the products that the store offers filtered by prices")
def get_products():
  """Get the products that the store offers"""
  # Connect with API
  response = requests.get("https://api.escuelajs.co/api/v1/products")
  products = response.json()
  return "".join([f"{product['title']} - {product['price']}" for product in products])


In [14]:
get_products.invoke({})

'Laptop - 1000Phone - 500Tablet - 300Monitor - 200'